# Demo of the Astronomy Commons Science Platform

Welcome to the Astronomy Commons Science Platform. Below is a demonstration of the utility and ease of use of Apache Spark and the Astronomy eXtensions to Spark (AXS) hosted on an elastic, cloud computing platform. Run the following cells in order to get started.

First, we created a `dirac` python module that wraps Apache Spark and AXS and uses the Kubernetes API to query the status of the Spark cluster used for computation.

In [ ]:
import dirac

Next, execute the following line to create a Spark cluster with 24 machines. These machines are requested on-demand from the cloud provider (Amazon Web Services). From our perspective, these machines do not exist and cost nothing until they you run the following line to request them.

In [ ]:
db = dirac.DataBase(conf={"spark.executor.instances": "24"})

After running this line, you will be given a "Dashboard" link that points you to the Apache Spark User Interface, an HTML webpage that shows the status of your cluster and any computations. You will also notice that a table has appeared showing several rows of "Pending" Kubernetes pods. This shows the status of the Spark executor machines as they come online. They won't be available for use until the "Pending" status changes to "Running." This can take up to 5 minutes to complete. Once it completes, execute the following cells.

The Spark cluster communicates with an Apache Hive metastore, which contains pointers to catalogs of data that are stored on Amazon S3. To find out what catalogs are available, run the following:

In [ ]:
catalogs = db.get_catalogs()
print(catalogs.list_table_names())

Next we can use the `.count()` function of Spark DataFrames to find how large these datasets are:

In [ ]:
for catalog in catalogs.list_table_names():
    print(catalog, "has", catalogs.load(catalog).count(), "rows")

These tables contain ~1 billion+ rows, yet we can perform computations relatively quickly! The corresponding data volumes are roughly:

|Catalog Name | Rows | Data Size (GB) |
|-----|-----------|-|
| allwise | 809,943,596 | 348.5 |
| gaia_source_edr3 | 2,113,651,513 | 959.7 |
| gaiadr2 | 1,833,997,578 | 420.9 |
| ps1 | 2,238,951,222 | 401.8 |
| sdss | 768,618,139 | 64.8 |
| ztf | 1,187,012,476 | 3,400 |

Next, let's take one of these catalogs and create a sky-map -- a 2D histogram of the locations of the objects on the sky. We'll use the Gaia DR3 catalog since Gaia is an all-sky survey. Execute the following, which loads Gaia DR3 and creates a new column called "hpix12" that is used for partitioning the sky in HEALPixels.

In [ ]:
import pyspark.sql.functions as sparkfunc

gaia_ = catalogs.load("gaia_source_edr3")
gaia = gaia_.withColumn("hpix12", sparkfunc.floor(gaia_['source_id']/34359738368))

Then, we will create a sky-map using one of the AXS built-in functions `healpix_hist`:

In [ ]:
%%time

healpix_map = gaia.exclude_duplicates().healpix_hist(NSIDE=512)

The previous cell should have only taken ~1 minute to histogram 2+ billion sources. Execute the following cell to plot the output:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import healpy as hp

fig = plt.figure(dpi=120, figsize=[8, 6])
hp.mollview(healpix_map, title="All objects", rot=(0, 0, 0), nest=True, norm='hist', fig=fig)
plt.show()
fig = plt.figure(dpi=120, figsize=[8, 6])
hp.mollview(healpix_map, title="All objects", rot=(180, 0, 0), nest=True, norm='hist', fig=fig)
plt.show()

Once code execution is done, we can stop the cluster:

In [ ]:
db.stop()

This also triggers the shutting down of the machines requested from the cloud provider after ~5 minutes. If you quickly re-run the notebook from the start, it will immediately start using those machines again, however if you wait 5+ minutes, the machines may need to be requested again.

The price of running queries like these can also be quite cheap: the cluster created for this example costs ~1.20 USD for an hour of usage. Assuming the notebook is run from start to finish in 5 minutes and the machines are reclaimed by the cloud provider after 5 minutes, then a single run of this notebook costs ~0.20 USD.